In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from azure_asr import recognize_from_microphone, transcribe_audio_to_text
import pymodel
import os
from tqdm.notebook import tqdm

In [3]:
_model, _tokenizer = pymodel.load_model()

In [4]:
def correct_asr(text, model=_model, tokenizer=_tokenizer):
    corrected = pymodel.correct(text, model, tokenizer)
    return corrected

In [5]:
import re

def remove_characters(line):
    return re.sub('[!@#$.,]', '', line)

def find_diff_words(str1, str2):
    str1 =remove_characters(str1)
    str2 =remove_characters(str2)

    words1 = str1.split()
    words2 = str2.split()
    diff_words = []
    for i in range(min((len(words1), len(words2)))):
        if words1[i].lower() != words2[i].lower():
            # print(words1[i], words2[i])
            diffs = [i, words1[i], words2[i]]
            diff_words.append(diffs)
    return diff_words

In [6]:
files = os.listdir('./data/audios/test')

df = []
for file in tqdm(files):
    audio_file_path = './data/audios/test/' + file
    if os.path.exists(audio_file_path):
        try:
            transcription = transcribe_audio_to_text(audio_file_path)
            corrected_transcription = correct_asr(transcription)
        except:
            print('error')

        df.append([audio_file_path, transcription, corrected_transcription])

  0%|          | 0/382 [00:00<?, ?it/s]

error


In [8]:
import pandas as pd

df = pd.DataFrame(df, columns=['file', 'azure_transcription', 'corrected_transcription'])

df.head()

,file,azure_transcription,corrected_transcription
0,./data/audios/test/1249120_44142156_100535941.wav,I have some terrible problems when I'm breathi...,I have some terrible problems when i'm breathi...
1,./data/audios/test/1249120_44142156_102100267.wav,I feel really sad all the time.,I feel really sad all the time.
2,./data/audios/test/1249120_44142156_102691000.wav,My knee feels weak and it gave away the other ...,My knee feels weak and it gave way the other d...
3,./data/audios/test/1249120_44142156_103207247.wav,Jismi cut infected or just healing?,i have a cut infected or just healing?
4,./data/audios/test/1249120_44142156_104815781.wav,What's the symptoms of appendicitis?,What's the symptoms of appendicitis?


In [45]:
recording = pd.read_csv('data/audios/recordings_info.csv', index_col='file_name')
# recording.index = recording['file']
recording.head()

,phrase,prompt
file_name,,
1249120_43453425_58166571.wav,When I remember her I feel down,Emotional pain
1249120_43719934_43347848.wav,When I carry heavy things I feel like breaking...,Hair falling out
1249120_43719934_53187202.wav,there is too much pain when i move my arm,Heart hurts
1249120_31349958_55816195.wav,My son had his lip pierced and it is swollen a...,Infected wound
1249120_43719934_82524191.wav,My muscles in my lower back are aching,Infected wound


In [46]:
correct = []
for file in df['file']:
    file = file.split('/')[-1] 

    if file in recording.index.to_list():
        correct.append(recording.loc[file]['phrase'])
    else:
        correct.append('NO LABEL EXISTS')

df['og_transcript'] = correct


In [50]:
import re

def remove_characters(line):
    return re.sub('[!@#$.,]', '', line)

def find_diff_words(str1, str2):
    str1 =remove_characters(str1)
    str2 =remove_characters(str2)

    words1 = str1.split()
    words2 = str2.split()
    diff_words = []
    for i in range(min((len(words1), len(words2)))):
        if words1[i].lower() != words2[i].lower():
            # print(words1[i], words2[i])
            diffs = [i, words1[i], words2[i]]
            diff_words.append(diffs)
    return diff_words

In [51]:
differnces = []
for lin1, lin2 in zip(df['corrected_transcription'], df['og_transcript']):
    differnces.append(find_diff_words(lin1, lin2))

In [52]:
df['differences'] = differnces

In [53]:
df.to_csv('./tests/tested3.csv', index=False)

In [ ]:
mic = recognize_from_microphone()